# Data Processing

## Data Summary
- Eurovision votes - votes.csv
    - Eurovisionworld fan website
    - ISO alpha-2 country codes
- Country codes
    - TODO: Find dataset
    - Can webscrape from official [ISO website](https://www.iso.org/obp/ui/#search)
    - Made country_codes_raw.csv from copy-pasting
    - Need ISO alpha-2 and ISO alpha-3
- Country capitals
    - TODO: Find dataset
- Religious demographic data
    - WRP_national.csv
    - ISO alpha-3 country codes
- GDP data
    - World_Bank_Data.csv
    - ISO alpha-3 country codes

# Utilities
- TODO: Move to .py files?

In [4]:
import pandas as pd

## Clean Country Codes Data

In [5]:
codes_raw = pd.read_csv("../data/country_codes_raw.csv")
codes_raw.head()

,name_fr,iso-alpha-2,iso-alpha-3,numeric
0,Afghanistan (l'),AF,AFG,4.0
1,NaN,NaN,NaN,NaN
2,Albanie (l'),AL,ALB,8.0
3,NaN,NaN,NaN,NaN
4,Algérie (l'),DZ,DZA,12.0


In [9]:
def get_clean_codes(codes_raw):
    codes = codes_raw.dropna()
    codes = codes[["iso-alpha-2", "iso-alpha-3"]]
    codes["iso-alpha-2"] = codes["iso-alpha-2"].str.lower()
    return codes

In [10]:
country_codes = get_clean_codes(codes_raw)
country_codes.head()

,iso-alpha-2,iso-alpha-3
0,af,AFG
2,al,ALB
4,dz,DZA
6,as,ASM
8,ad,AND


In [11]:
countries = pd.read_csv("../data/country_info.csv")
countries.head()

,name,code,capital
0,Albania,al,Tirana
1,Armenia,am,Yerevan
2,Australia,au,Canberra
3,Austria,at,Vienna
4,Azerbaijan,az,Baku


In [14]:
merged_countries = countries.merge(country_codes, left_on="code", right_on="iso-alpha-2")
merged_countries = merged_countries.drop("code", axis=1)
merged_countries.head()

,name,capital,iso-alpha-2,iso-alpha-3
0,Albania,Tirana,al,ALB
1,Armenia,Yerevan,am,ARM
2,Australia,Canberra,au,AUS
3,Austria,Vienna,at,AUT
4,Azerbaijan,Baku,az,AZE


## GDP Statistics

In [45]:
gdp_raw = pd.read_csv("../data/World_Bank_Data.csv")
gdp_raw

,Series Name,Series Code,Country Name,Country Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],...,2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023]
0,"Population, total",SP.POP.TOTL,Afghanistan,AFG,9035043,9214083,9404406,9604487,9814318,10036008,...,32792523,33831764,34700612,35688935,36743039,37856121,39068979,40000412,40578842,41454761
1,"Population, total",SP.POP.TOTL,Algeria,DZA,11424922,11628883,11800771,11982118,12179813,12365976,...,39205031,40019529,40850721,41689299,42505035,43294546,44042091,44761099,45477389,46164219
2,"Population, total",SP.POP.TOTL,Angola,AGO,5231654,5301583,5354310,5408320,5464187,5521981,...,27160769,28157798,29183070,30234839,31297155,32375632,33451132,34532429,35635029,36749906
3,"Population, total",SP.POP.TOTL,Antigua and Barbuda,ATG,55603,56540,57336,58138,59020,59970,...,88765,89409,89969,90468,90926,91364,91846,92349,92840,93316
4,"Population, total",SP.POP.TOTL,Argentina,ARG,20386045,20726276,21072538,21421705,21769453,22112629,...,43024071,43477012,43900313,44288894,44654882,44973465,45191965,45312281,45407904,45538401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9053,Data from database: World Development Indicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
gdp_data = gdp_raw.drop(["Country Name", "Series Code"], axis=1)
gdp_data = gdp_data.iloc[:-5]
# gdp_data.unstack()
gdp_data = gdp_data.set_index(["Country Code","Series Name"])
gdp_data = gdp_data.stack().unstack(1)
# gdp_data["Series Name"].unique()
# gdp_raw["Series Name"].unique()

gdp_data = gdp_data[['GDP per capita (current US$)', 'Population, total']].reset_index()
gdp_data["Year"] = gdp_data["level_1"].str.split().str.get(0)
gdp_data["Year"] = pd.to_numeric(gdp_data["Year"])
gdp_data = gdp_data.drop("level_1", axis=1)
gdp_data
# gdp_data[gdp_data["Country Code"] == "DEU"]

Series Name,Country Code,GDP per capita (current US$),"Population, total",Year
0,ABW,..,54922,1960
1,ABW,..,55578,1961
2,ABW,..,56320,1962
3,ABW,..,57002,1963
4,ABW,..,57619,1964
...,...,...,...,...
11579,ZWE,1684.02790388182,15271368,2019
11580,ZWE,1730.41348946953,15526888,2020
11581,ZWE,1724.38773104902,15797210,2021
11582,ZWE,2040.55245910726,16069056,2022
